### Importing packages and data_utils.py

In [1]:
from climsim_utils.data_utils import *
import downloads

### Download Data from HuggingFace

In [3]:
downloads.download_grid_info()
downloads.download_normalizations()
downloads.fetch_train_data()

File already exists in ../grid_info/. Skipping download.
File already exists in ../preprocessing/normalizations/. Skipping download.
File already exists in ../preprocessing/normalizations/. Skipping download.
File already exists in ../preprocessing/normalizations/. Skipping download.
File already exists in ../preprocessing/normalizations/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File alre

### Instantiating class

In [5]:
grid_path = '../grid_info/ClimSim_low-res_grid-info.nc'
norm_path = './normalizations/'

grid_info = xr.open_dataset(grid_path)
input_mean = xr.open_dataset(norm_path + 'inputs/input_mean.nc')
input_max = xr.open_dataset(norm_path + 'inputs/input_max.nc')
input_min = xr.open_dataset(norm_path + 'inputs/input_min.nc')
output_scale = xr.open_dataset(norm_path + 'outputs/output_scale.nc')

data = data_utils(grid_info = grid_info, 
                  input_mean = input_mean, 
                  input_max = input_max, 
                  input_min = input_min, 
                  output_scale = output_scale)

### Create training data

In [6]:
# set data path
data.data_path = '../e3sm_train/'

# set inputs and outputs to V1 subset
data.set_to_v1_vars()

# v1 inputs (name :: description :: dimension :: units): 

# 'state_t' :: air temperature :: 60 :: K 
# 'state_q0001' :: specific humidity :: 60 :: kg/kg
# 'state_ps' :: surface pressure :: 1 :: Pa
# 'pbuf_SOLIN' :: solar insolation :: 1 :: W/m^2
# 'pbuf_LHFLX' :: surface latent heat flux :: 1 :: W/m^2
# 'pbuf_SHFLX' :: surface sensible heat flux :: 1 :: W/m^2

# v1 outputs (name :: description :: dimension :: units): 

# 'ptend_t' :: heating tendency :: 60 :: K/s 
# 'ptend_q0001' :: moistening tendency :: 60 :: kg/kg/s
# 'cam_out_NETSW' :: net shortwave flux at surface :: 1 :: W/m^2
# 'cam_out_FLWDS' :: downward longwave flux at surface :: 1 :: W/m^2 
# 'cam_out_PRECSC' :: snow rate (liquid water equivalent) :: 1 :: m/s 
# 'cam_out_PRECC' :: rain rate :: 1 :: m/s
# 'cam_out_SOLS' :: downward visible direct solar flux to surface :: 1 :: W/m^2
# 'cam_out_SOLL' :: downward near-infrared direct solar flux to surface :: 1 :: W/m^2
# 'cam_out_SOLSD' :: downward diffuse solar flux to surface :: 1 :: W/m^2
# 'cam_out_SOLLD' :: downward diffuse near-infrared solar flux to surface :: 1 :: W/m^2

# set regular expressions for selecting training data
data.set_regexps(data_split = 'train', 
                 regexps = ['E3SM-MMF.mli.000[1234567]-*-*-*.nc', # years 1 through 7
                            'E3SM-MMF.mli.0008-01-*-*.nc']) # first month of year 8
# set temporal subsampling
data.set_stride_sample(data_split = 'train', stride_sample = 7)
# create list of files to extract data from
data.set_filelist(data_split = 'train')
# save numpy files of training data
data.save_as_npy(data_split = 'train', save_path = '')

2023-06-14 03:20:58.359697: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


### Create validation data

In [7]:
# set regular expressions for selecting validation data
data.set_regexps(data_split = 'val',
                 regexps = ['E3SM-MMF.mli.0008-0[23456789]-*-*.nc', # months 2 through 9 of year 8
                            'E3SM-MMF.mli.0008-1[012]-*-*.nc', # months 10 through 12 of year 8
                            'E3SM-MMF.mli.0009-01-*-*.nc']) # first month of year 9
# set temporal subsampling
data.set_stride_sample(data_split = 'val', stride_sample = 7)
# create list of files to extract data from
data.set_filelist(data_split = 'val')
# save numpy files of validation data
data.save_as_npy(data_split = 'val', save_path = '')

2023-06-14 05:19:01.725068: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


### Create scoring data

In [8]:
# set regular expressions for selecting scoring data (stride of 6 is needed for daily averaging)
data.set_regexps(data_split = 'scoring',
                 regexps = ['E3SM-MMF.mli.0008-0[23456789]-*-*.nc', # months 2 through 9 of year 8
                            'E3SM-MMF.mli.0008-1[012]-*-*.nc', # months 10 through 12 of year 8
                            'E3SM-MMF.mli.0009-01-*-*.nc']) # first month of year 9
# set temporal subsampling
data.set_stride_sample(data_split = 'scoring', stride_sample = 6)
# create list of files to extract data from
data.set_filelist(data_split = 'scoring')
# save numpy files of scoring data
data.save_as_npy(data_split = 'scoring', save_path = '')

2023-06-14 05:35:01.140716: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [9]:
print('finished')

finished
